In [1]:
import pandas as pd 
import dask.dataframe as dd
# import pyarrow as pa
import numpy as np
import tqdm
from datetime import datetime as dt
# from tsfresh.feature_extraction import extract_features
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot
import scipy
import seaborn
import jieba
# jieba.enable_parallel(4)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.preprocessing import StandardScaler
from tsfresh import extract_features 

import lightgbm as lgbm

In [2]:
collection = MongoClient('52.68.95.18:27017')['BTCUSD_order_book']['binance_order_book']

In [3]:
order_book_df = pd.DataFrame(list(collection.find({})))

In [4]:
order_book_df.head(3)

,_id,asks,bids,buy_ratio,buy_volume,datetime,last_price,market_pair,sell_ratio,sell_volume,timestamp,total_buy_btc,total_sell_btc,updatetime
0,5d25872298c3b168ff54531a,"[[12978.36000000, 0.03030600], [12979.76000000...","[[12976.66000000, 0.10164700], [12976.64000000...",30.97667173765498,1.316381e+07,2019-07-10 14:35:11,12978.36000000,BTCUSD,0.03228235778424215,426686.081657,1562740511,1016.818264,32.825291,2019-07-10 14:35:13
1,5d25881298c3b168ff54539a,"[[12968.46000000, 0.13648400], [12968.47000000...","[[12967.35000000, 0.03035800], [12967.33000000...",32.04501619267269,1.283020e+07,2019-07-10 14:39:11,12967.35000000,BTCUSD,0.031206100630046075,401769.292752,1562740751,991.114025,30.928804,2019-07-10 14:39:13
2,5d25882398c3b168ff5453bc,"[[12969.00000000, 0.39608300], [12969.98000000...","[[12967.38000000, 1.00000000], [12967.37000000...",29.928371958159985,1.288608e+07,2019-07-10 14:39:29,12969.00000000,BTCUSD,0.033413110522617306,432191.347667,1562740769,995.424056,33.260214,2019-07-10 14:39:29


In [5]:
order_book_df.drop(['market_pair', 'datetime', 'timestamp'], axis=1 , inplace=True)

In [6]:
ticker_collection = MongoClient('52.68.95.18:27017')['BTCUSD_order_book']['binance_ticker']

In [7]:
ticker_df = pd.DataFrame(list(ticker_collection.find({})))

In [8]:
for column in ['askPrice', 'askQty', 'bidPrice', 'bidQty']:
    ticker_df[column] = ticker_df[column].astype(float)

In [9]:
ticker_df.head(3)

,_id,askPrice,askQty,bidPrice,bidQty,closeTime,count,datetime,firstId,highPrice,...,openTime,prevClosePrice,priceChange,priceChangePercent,quoteVolume,symbol,timestamp,updatetime,volume,weightedAvgPrice
0,5d25872298c3b168ff545312,12978.36,0.031418,12976.66,0.101647,1562740514133,707346,2019-07-10 14:35:11,151090107,13110.10000000,...,1562654114133,12682.42000000,297.05000000,2.343,1064347327.48953093,BTCUSDT,1562740511,2019-07-10 14:35:13,84667.66264700,12570.88354886
1,5d25881298c3b168ff545392,12968.46,0.136484,12967.35,0.030358,1562740753480,706286,2019-07-10 14:39:11,151092126,13110.10000000,...,1562654353480,12660.35000000,307.00000000,2.425,1063256693.22454173,BTCUSDT,1562740751,2019-07-10 14:39:13,84579.36503600,12571.11226565
2,5d25882298c3b168ff5453b4,12969.00,0.396083,12967.38,1.000000,1562740770433,706263,2019-07-10 14:39:29,151092192,13110.10000000,...,1562654370433,12670.82000000,300.24000000,2.370,1063251345.09378180,BTCUSDT,1562740769,2019-07-10 14:39:29,84578.87919500,12571.12124461


In [10]:
ticker_df.drop(['market_pair', 'symbol', 'timestamp', 'datetime','closeTime','count','firstId','lastId','openTime','weightedAvgPrice'], axis=1, inplace=True)

In [11]:
ticker_df['askBidMean'] = (ticker_df['askPrice'] + ticker_df['bidPrice'])/2

In [12]:
train_df = pd.merge(order_book_df, ticker_df, on='updatetime')

In [13]:
train_df['datetime_hour'] = train_df['updatetime'].apply(lambda x: dt.strftime(dt.strptime(x,"%Y-%m-%d %H:%M:%S"),"%Y-%m-%d %H"))

In [14]:
for column in ['buy_ratio', 'buy_volume', 'last_price',
       'sell_ratio', 'sell_volume', 'total_buy_btc', 'total_sell_btc',
       'askPrice', 'askQty', 'bidPrice', 'bidQty',
       'highPrice', 'lastPrice', 'lastQty', 'lowPrice', 'openPrice',
       'prevClosePrice', 'priceChange', 'priceChangePercent', 'quoteVolume',
       'volume', 'askBidMean']:
    train_df[column] = train_df[column].astype(float)

In [15]:
groupby = train_df.groupby("datetime_hour").mean().add_prefix('mean_')

In [16]:
groupby1 = train_df.groupby("datetime_hour").max().drop(['asks', 'bids', 'updatetime','_id_y', '_id_x'], axis=1).add_prefix('max_')

In [17]:
groupby2 = train_df.groupby("datetime_hour").min().drop(['asks', 'bids', 'updatetime','_id_y', '_id_x'], axis=1).add_prefix('min_')

In [18]:
groupby3 = train_df.groupby("datetime_hour").skew().add_prefix('skew_')

In [19]:
groupby4 = train_df.groupby("datetime_hour").median().add_prefix('median_')

In [20]:
groupby5 = train_df.groupby("datetime_hour").std().add_prefix('std_')

In [21]:
train_df.columns

Index(['_id_x', 'asks', 'bids', 'buy_ratio', 'buy_volume', 'last_price',
       'sell_ratio', 'sell_volume', 'total_buy_btc', 'total_sell_btc',
       'updatetime', '_id_y', 'askPrice', 'askQty', 'bidPrice', 'bidQty',
       'highPrice', 'lastPrice', 'lastQty', 'lowPrice', 'openPrice',
       'prevClosePrice', 'priceChange', 'priceChangePercent', 'quoteVolume',
       'volume', 'askBidMean', 'datetime_hour'],
      dtype='object')

In [446]:
fcp = {
        'fft_coefficient': [{'coeff': 0, 'attr': 'abs'}],
        'absolute_sum_of_changes':None,
        'standard_deviation':None,
        'minimum':None
#         'kurtosis':None,
#         'skewness':None,
#         'median':None,
#         'number_peaks' : [{'n':10}],
#         'variance_larger_than_standard_deviation':None,
#         'count_above_mean':None,
#         'longest_strike_above_mean':None
}
aggdfts = extract_features(train_df, column_id='datetime_hour', column_sort='updatetime',column_value = 'total_sell_btc',default_fc_parameters=fcp,n_jobs=2)
aggdfts2 = extract_features(train_df, column_id='datetime_hour', column_sort='updatetime',column_value = 'total_buy_btc',default_fc_parameters=fcp,n_jobs=2)
aggdfts3 = extract_features(train_df, column_id='datetime_hour', column_sort='updatetime',column_value = 'volume',default_fc_parameters=fcp,n_jobs=2)
# aggdfts4 = extract_features(train_df, column_id='datetime_hour', column_sort='updatetime',column_value = 'askBidMean',default_fc_parameters=fcp,n_jobs=2)
aggdfts5 = extract_features(train_df, column_id='datetime_hour', column_sort='updatetime',column_value = 'sell_volume',default_fc_parameters=fcp,n_jobs=2)
aggdfts6 = extract_features(train_df, column_id='datetime_hour', column_sort='updatetime',column_value = 'buy_volume',default_fc_parameters=fcp,n_jobs=2)
aggdfts7 = extract_features(train_df, column_id='datetime_hour', column_sort='updatetime',column_value = 'buy_ratio',default_fc_parameters=fcp,n_jobs=2)
aggdfts8 = extract_features(train_df, column_id='datetime_hour', column_sort='updatetime',column_value = 'sell_ratio',default_fc_parameters=fcp,n_jobs=2)


Feature Extraction: 100%|██████████| 10/10 [00:00<00:00, 146.91it/s]

In [447]:
y = train_df.groupby("datetime_hour").tail(10).groupby("datetime_hour").mean()['askBidMean']

In [448]:
cleanData = pd.concat([groupby],axis=1)
# cleanData = cleanData.join(aggdfts, on='datetime_hour')
# cleanData = cleanData.join(aggdfts2, on='datetime_hour')
# cleanData = cleanData.join(aggdfts5, on='datetime_hour')
# cleanData = cleanData.join(aggdfts6, on='datetime_hour')
# cleanData = cleanData.join(aggdfts7, on='datetime_hour')
cleanData = cleanData.join(aggdfts8, on='datetime_hour')
# cleanData = groupby.drop(['asks', 'bids', '_id_y', 'updatetime','_id_x'], axis=1)
# cleanData = cleanData.drop(['asks', 'bids', 'updatetime','_id_y', '_id_x'], axis=1)

In [449]:
x_window, y_window = 1, 1

In [450]:
x = np.zeros((len(cleanData)-x_window-y_window+1,x_window, len(cleanData.columns)))

In [451]:
y_max = np.zeros(len(cleanData)-x_window-y_window+1)
y_min = np.zeros(len(cleanData)-x_window-y_window+1)
y_mean = np.zeros(len(cleanData)-x_window-y_window+1)
y_ma = np.zeros(len(cleanData)-x_window-y_window+1)

In [452]:
cleanDataX = np.asarray(cleanData)
cleanDataY = np.asarray(y)

In [453]:
# scaler = StandardScaler()
# scaler.fit(cleanDataX)
# cleanDataX = scaler.transform(cleanDataX)

In [454]:
for index, i in enumerate(cleanDataX):
    x[index] = cleanDataX[index:index+x_window]
    y_mean[index] = cleanDataY[(index+x_window+y_window-1):(index+x_window+y_window)].mean()
    y_max[index] = cleanDataY[(index+x_window): (index+x_window+y_window)].max()
    y_min[index] = cleanDataY[(index+x_window): (index+x_window+y_window)].min()
    if index+x_window+y_window == len(cleanDataX):
        break

In [455]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Activation, Flatten, Dropout, GRU, Input
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.models import Model

In [456]:
stratified_folder = KFold(n_splits=4, random_state=42, shuffle=False) 
for train_index, test_index in stratified_folder.split(x,y_mean): 
    X_train, X_test = x[train_index].reshape(x[train_index].shape[0],-1), x[test_index].reshape(x[test_index].shape[0],-1)
    y_train, y_test = y_mean[train_index], y_mean[test_index] 
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'mse',
        'subsample': .7,
        'colsample_bytree': .5,
        'reg_alpha': .1,
        'reg_lambda': .1,
        'min_split_gain': 0.01,
        'min_child_weight': 1,
        'silent':True,
        'verbosity':-1,
        'n_jobs':3
    }

    clf = lgbm.LGBMRegressor(**lgb_params,learning_rate=0.02,
                                    n_estimators=1000,max_depth=2)
    clf.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train),(X_test, y_test)],
        verbose=-1,
        eval_metric='rmse',
        early_stopping_rounds=100
    )
    pred = clf.predict(X_test, num_iteration=clf.best_iteration_)
    print("predict ")
    print(pred)
    print("true ")
    print(y_test)
#     print(f1_score(pred, y_test, average='macro'))
#     print(confusion_matrix(pred, y_test))

Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 78.8143	valid_0's l2: 6211.69	valid_1's rmse: 618.548	valid_1's l2: 382601
predict 
[11743.46872349 11766.70694402 11705.82144763 11696.51070062
 11701.90217675 11722.01555347 11815.84529318 11713.39365716
 11707.23507523 11736.29747903 11603.91712011 11597.94140978
 11551.49092377 11578.17289097 11556.88055619 11501.50450595
 11608.25621588 11577.6650861  11394.9237155  11398.98417389
 11409.07249992 11475.31490519 11314.91448442 11203.46319914
 11351.86655017 11355.77902606 11345.99551154 11321.11002599
 11304.82097083 11373.41947919 11370.31283699 11420.39715405
 11469.35697648 11386.17436843 11530.86227885 11436.40339049
 11515.65308482 11314.28701854 11187.1542175  11275.6800114
 11276.38091766 11369.20894685]
true 
[12869.488  12943.3    12955.445  12965.6785 12981.663  13007.382
 13034.8565 12726.364  12381.024  12245.3145 12281.877  12218.9445
 1

In [145]:
stratified_folder = KFold(n_splits=4, random_state=0, shuffle=False) 
for train_index, test_index in stratified_folder.split(x,y_mean): 
    X_train, X_test = x[train_index], x[test_index]
    y_train_mean, y_test_mean = y_mean[train_index], y_mean[test_index]
    y_train_max, y_test_max = y_max[train_index], y_max[test_index] 
    y_train_min, y_test_min = y_min[train_index], y_min[test_index] 
    
    input_layer = Input(shape=(X_train.shape[1],X_train.shape[2]))
    hidden_layer = GRU(128, return_sequences=True, activation='tanh')(input_layer)
    hidden_layer = Flatten()(hidden_layer)
    
    mean_hidden_layer = Dense(2048, activation='relu')(hidden_layer)
    mean_hidden_layer = Dropout(0.1)(mean_hidden_layer)
    
#     mean_hidden_layer = Dense(2048, activation='relu')(mean_hidden_layer)
#     mean_hidden_layer = Dropout(0.1)(mean_hidden_layer)
#     max_hidden_layer = Dense(256, activation='relu')(hidden_layer)
#     max_hidden_layer = Dropout(0.1)(max_hidden_layer)
    
#     min_hidden_layer = Dense(256, activation='relu')(hidden_layer)
#     min_hidden_layer = Dropout(0.1)(min_hidden_layer)
    
    output_layer_1 = Dense(1,activation='linear')(mean_hidden_layer)
#     output_layer_2 = Dense(1,activation='linear')(max_hidden_layer)
#     output_layer_3 = Dense(1,activation='linear')(min_hidden_layer)
    
    opt = Adam(lr=0.01)
    
    ml_model = Model(inputs=input_layer, outputs=[output_layer_1])
    ml_model.compile(optimizer=opt, loss='mse')
    ml_model.fit(X_train, [y_train_mean], epochs=200, verbose=1, batch_size=8192)
    
    pred = ml_model.predict(X_test, batch_size=256)
#     break
#     print(f1_score(pred, y_test, average='macro'))
#     print(confusion_matrix(pred, y_test))

Epoch 1/200
122/122 [==============================] - 2s 13ms/step - loss: 117800824.0000
Epoch 2/200
122/122 [==============================] - 0s 236us/step - loss: 116983696.0000
Epoch 3/200
122/122 [==============================] - 0s 317us/step - loss: 115967544.0000
Epoch 4/200
122/122 [==============================] - 0s 300us/step - loss: 114643344.0000
Epoch 5/200
122/122 [==============================] - 0s 405us/step - loss: 113103672.0000
Epoch 6/200
122/122 [==============================] - 0s 495us/step - loss: 111123328.0000
Epoch 7/200
122/122 [==============================] - 0s 521us/step - loss: 108785352.0000
Epoch 8/200
122/122 [==============================] - 0s 392us/step - loss: 106158384.0000
Epoch 9/200
122/122 [==============================] - 0s 439us/step - loss: 103153448.0000
Epoch 10/200
122/122 [==============================] - 0s 457us/step - loss: 99870168.0000
Epoch 11/200
122/122 [==============================] - 0s 334us/step - loss: 961

KeyboardInterrupt: 